# DenseNet121 Testing

In [1]:
from keras.preprocessing.image import ImageDataGenerator

# Create image augmenter
dataAugment = ImageDataGenerator(rescale=1.0/255,
                                 rotation_range=40,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 zoom_range=0.2,
                                 shear_range=0.2,
                                 fill_mode='reflect',
                                 horizontal_flip=True,
                                 validation_split=0.2,
                                 data_format='channels_last'
                                )
        
trainDataGenerator = dataAugment.flow_from_directory(directory='Data/Train',
                                                     target_size=(224, 224), # change this to the input size used for your model
                                                     class_mode='categorical',
                                                     subset='training',
                                                     shuffle=True
                                                     # batch_size=
                                                    )

validationDataGenerator = dataAugment.flow_from_directory(directory='Data/Train',
                                                          target_size=(224,224), # change this to the input size used for your model
                                                          class_mode='categorical',
                                                          subset='validation',
                                                          shuffle=True
                                                          # batch_size=
                                                         )

Using TensorFlow backend.


Found 221 images belonging to 3 classes.
Found 54 images belonging to 3 classes.


In [4]:
from keras.applications.densenet import DenseNet121
DenseNet = DenseNet121(input_shape=(224,224,3),include_top=False, weights='imagenet')
DenseNet.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_5 (ZeroPadding2D (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_5[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

conv4_block1_1_bn (BatchNormali (None, 14, 14, 128)  512         conv4_block1_1_conv[0][0]        
__________________________________________________________________________________________________
conv4_block1_1_relu (Activation (None, 14, 14, 128)  0           conv4_block1_1_bn[0][0]          
__________________________________________________________________________________________________
conv4_block1_2_conv (Conv2D)    (None, 14, 14, 32)   36864       conv4_block1_1_relu[0][0]        
__________________________________________________________________________________________________
conv4_block1_concat (Concatenat (None, 14, 14, 288)  0           pool3_pool[0][0]                 
                                                                 conv4_block1_2_conv[0][0]        
__________________________________________________________________________________________________
conv4_block2_0_bn (BatchNormali (None, 14, 14, 288)  1152        conv4_block1_concat[0][0]        
__________

__________________________________________________________________________________________________
conv5_block1_concat (Concatenat (None, 7, 7, 544)    0           pool4_pool[0][0]                 
                                                                 conv5_block1_2_conv[0][0]        
__________________________________________________________________________________________________
conv5_block2_0_bn (BatchNormali (None, 7, 7, 544)    2176        conv5_block1_concat[0][0]        
__________________________________________________________________________________________________
conv5_block2_0_relu (Activation (None, 7, 7, 544)    0           conv5_block2_0_bn[0][0]          
__________________________________________________________________________________________________
conv5_block2_1_conv (Conv2D)    (None, 7, 7, 128)    69632       conv5_block2_0_relu[0][0]        
__________________________________________________________________________________________________
conv5_bloc

In [5]:


for layer in DenseNet.layers[:-7]:
        layer.trainable = False


# Check the trainable status of the individual layers
for layer in DenseNet.layers:
    if (layer.trainable == True):
        print(layer, layer.trainable)

(<keras.layers.convolutional.Conv2D object at 0xb4db26710>, True)
(<keras.layers.normalization.BatchNormalization object at 0xb4db3ac50>, True)
(<keras.layers.core.Activation object at 0xb4dbc1c50>, True)
(<keras.layers.convolutional.Conv2D object at 0xb4dbdf6d0>, True)
(<keras.layers.merge.Concatenate object at 0xb4dc10b90>, True)
(<keras.layers.normalization.BatchNormalization object at 0xb4c057650>, True)
(<keras.layers.core.Activation object at 0xb4dc10d50>, True)


In [6]:
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import optimizers, models


model = models.Sequential()

model.add(DenseNet)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='nadam',metrics=['accuracy'])


In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 7, 7, 1024)        7037504   
_________________________________________________________________
flatten_1 (Flatten)          (None, 50176)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               12845312  
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 771       
Total params: 19,883,587
Trainable params: 13,012,227
Non-trainable params: 6,871,360
_________________________________________________________________


In [8]:
model.fit_generator(generator=trainDataGenerator,
                    steps_per_epoch=trainDataGenerator.samples/trainDataGenerator.batch_size,
                    epochs=10,
                    validation_data=validationDataGenerator,
                    validation_steps=validationDataGenerator.samples/validationDataGenerator.batch_size,
                    verbose=1
                   )

Epoch 1/10


KeyboardInterrupt: 